In [27]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Load the dataset into PySpark
spark = SparkSession.builder.appName("craigslist_vehicles").getOrCreate()
data = spark.read.csv("data/craigslist_vehicles.csv", header=True, inferSchema=True)

# Drop the unnecessary columns
columns_to_drop = ['Unnamed: 0', 'url', 'region_url', 'VIN', 'image_url', 'description', 'county', 'lat', 'long', 'removal_date']
data = data.drop(*columns_to_drop)

# Convert the posting_date column to a datetime type
data = data.withColumn("posting_date", F.to_date(data["posting_date"]))

In [28]:
# check if columns exist before dropping
columns_to_drop = ['Unnamed: 0', 'url', 'region_url', 'VIN', 'image_url', 'description', 'county', 'lat', 'long', 'removal_date']

# filter the columns to drop only those that exist in the DataFrame
columns_to_drop_existing = [col for col in columns_to_drop if col in data.columns]

# drop the existing columns
data = data.drop(columns=columns_to_drop_existing)


# convert 'posting_date' to datetime data type
data['posting_date'] = pd.to_datetime(data['posting_date'],  utc=True)


#preview the first 5 rows of 'craigslist_vehicles.csv' dataset after droping unncessary columns and coverting "post)time" to date. 
data.head()

TypeError: drop() got an unexpected keyword argument 'columns'